Pre-processing the NICT JLE dataset

In [1]:
import re
from os import listdir


def generateCleanFile(filename):
    input = open("NICT_JLE_4.1/LearnerOriginal/" + filename, "r", encoding="latin1") 
    output = open("preProcessedData/" + filename, "w", encoding="latin1")
    lines = input.readlines()
    for line in lines:
        if '<B>' in line :
            output.write(removeInternalTags(line[3:len(line)-5])+"\n")
    input.close()
    output.close() 

# TODO: Keep signaficant tags that will help with the prediction of the SST level of each participant 
def removeInternalTags(line):
    doubleTagPattern = "<.*?>(.+?)</.*?>"
    singleTagPattern = "</?.*?>"
    line = re.sub(doubleTagPattern, "", line)
    line = re.sub(singleTagPattern, "", line)
    return re.sub(r"\s+", " ", line)

files = [f for f in listdir("NICT_JLE_4.1/LearnerOriginal")]

for f in files:
    generateCleanFile(f)


Processing the dataset: extract features with the Bag of Word

In [16]:
import nltk
from nltk.corpus import stopwords
from os import listdir
import numpy
import re


def word_extraction(sentence):    
    ignore = set(stopwords.words('english'))   
    words = re.sub("[^\w]", " ",  sentence).split()    
    cleaned_text = [w.lower() for w in words if w not in ignore]    
    return cleaned_text

def tokenize(sentences):    
    words = []    
    for sentence in sentences:        
        w = word_extraction(sentence)        
        words.extend(w)            
        words = sorted(list(set(words)))    
    return words

def generate_bow(filename):
    f = open("preProcessedData/" + filename, "r", encoding="latin1") 
    lines = f.readlines()
    vocab = tokenize(lines)
    print("vocab",len(vocab))
    vector = list()        
    for line in lines:        
        words = word_extraction(line)        
        bag_vector = numpy.zeros(len(vocab))        
        for w in words:            
            for i,word in enumerate(vocab):                
                if word == w:                     
                    bag_vector[i] += 1 
        print(len(bag_vector))         
    f.close()                 
    return vector

    

files = [f for f in listdir("preProcessedData")]

# TODO : save the BoW output in a variable as our input for the classifier 


vocab 168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
[]


Training the classifier to Predict SST

Accuracy and Confusion matrix